In [6]:
import os
import tensorflow as tf

# Caminho do modelo
tflite_path = "mad_model_multisensors_float32_notimestamp.tflite"

# Remove o modelo anterior, se existir
if os.path.exists(tflite_path):
    os.remove(tflite_path)
    print("Modelo antigo removido.")

# Classe do modelo com múltiplos sensores (sem timestamp)
class MultiSensorMADModel(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec([None, 3, None], tf.float32)])  # x, y, z
    def __call__(self, inputs):
        def compute_mad(sensor_data):
            x = sensor_data[0]
            y = sensor_data[1]
            z = sensor_data[2]

            mag = tf.sqrt(x * x + y * y + z * z)

            mean = tf.reduce_mean(mag)
            std_dev = tf.math.reduce_std(mag)
            min_val = tf.reduce_min(mag)
            max_val = tf.reduce_max(mag)

            return tf.stack([mean, std_dev, min_val, max_val])

        results = tf.map_fn(compute_mad, inputs, fn_output_signature=tf.float32)
        return results

# Instancia e exporta o modelo
mad_model = MultiSensorMADModel()
concrete_fn = mad_model.__call__.get_concrete_function(
    tf.TensorSpec(shape=[None, 3, None], dtype=tf.float32)  # [num_sensores, 3, N]
)

# Converte para TFLite
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_fn], mad_model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()

# Salva o modelo
with open(tflite_path, "wb") as f:
    f.write(tflite_model)
    print("Novo modelo float32 (sem timestamp) salvo com sucesso.")

# Verificação
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()

print("✅ Verificação do modelo:")
print("Shape de entrada:", input_details[0]['shape'])
print("Shape signature :", input_details[0]['shape_signature'])
print("Tipo de entrada :", input_details[0]['dtype'])

Modelo antigo removido.
INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmps_59zebd/assets


INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmps_59zebd/assets


Novo modelo float32 (sem timestamp) salvo com sucesso.
✅ Verificação do modelo:
Shape de entrada: [1 3 1]
Shape signature : [-1  3 -1]
Tipo de entrada : <class 'numpy.float32'>


W0000 00:00:1750688809.240469 22586913 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1750688809.240545 22586913 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-06-23 11:26:49.241232: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmps_59zebd
2025-06-23 11:26:49.241956: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-23 11:26:49.241969: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmps_59zebd
2025-06-23 11:26:49.246426: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-06-23 11:26:49.254252: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmps_59zebd
2025-06-23 11:26:49.258255: I tensorflow/cc/saved_model/loader.c

In [7]:
import numpy as np
import tensorflow as tf
import time
import os

# --- Função de referência atualizada (sem timestamp) ---
def get_mad_py(data):
    # data: shape [3, N] -> [x, y, z]
    magnitudes = np.sqrt(np.sum(np.square(data), axis=0))  # shape [N]
    mean = np.mean(magnitudes)
    std_dev = np.std(magnitudes)
    min_val = np.min(magnitudes)
    max_val = np.max(magnitudes)
    return np.array([mean, std_dev, min_val, max_val], dtype=np.float32)

# --- Geração de dados simulados ---
num_sensors = 32
N = 1_000_000

multi_sensor_data = []
for _ in range(num_sensors):
    x = np.random.rand(N).astype(np.float32) * 10
    y = np.random.rand(N).astype(np.float32) * 10
    z = np.random.rand(N).astype(np.float32) * 10
    sensor_data = np.stack([x, y, z], axis=0)  # shape [3, N]
    multi_sensor_data.append(sensor_data)

multi_sensor_input = np.stack(multi_sensor_data, axis=0)  # shape [num_sensors, 3, N]

# --- Tempo: Cálculo Python puro ---
start_cpu = time.time()
mad_refs = [get_mad_py(sensor) for sensor in multi_sensor_data]
mad_refs = np.stack(mad_refs)  # shape [num_sensors, 4]
end_cpu = time.time()
cpu_time_ms = (end_cpu - start_cpu) * 1000

# --- Tempo: Carregamento do modelo TFLite ---
model_path = "mad_model_multisensors_float32_notimestamp.tflite"
start_load = time.time()
interpreter = tf.lite.Interpreter(model_path=model_path)
end_load = time.time()
load_time_ms = (end_load - start_load) * 1000

# --- Ajuste de entrada ---
interpreter.resize_tensor_input(0, [num_sensors, 3, N])
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# --- Inferência ---
start_infer = time.time()
interpreter.set_tensor(input_details[0]['index'], multi_sensor_input)
interpreter.invoke()
mad_out = interpreter.get_tensor(output_details[0]['index'])  # shape [num_sensors, 4]
end_infer = time.time()
tflite_time_ms = (end_infer - start_infer) * 1000

# --- Comparação por sensor ---
print("📊 Comparação por sensor:")
for i in range(num_sensors):
    print(f"\n🔹 Sensor {i+1}")
    print("Referência (Python): ", mad_refs[i])
    print("Modelo TFLite      : ", mad_out[i])
    print("Erro absoluto      : ", np.abs(mad_refs[i] - mad_out[i]))

# --- Estatísticas gerais ---
model_size_kb = os.path.getsize(model_path) / 1024
input_tensor_bytes = multi_sensor_input.nbytes / 1024
reduction_pct = ((cpu_time_ms - tflite_time_ms) / cpu_time_ms) * 100

print(f"\n⏱️ Tempo CPU total (Python):        {cpu_time_ms:.3f} ms")
print(f"📦 Tempo de carga TFLite:           {load_time_ms:.3f} ms")
print(f"⚡ Tempo de inferência TFLite:      {tflite_time_ms:.3f} ms")
print(f"📉 Redução de tempo (inferência):  {reduction_pct:.2f}%")
print(f"💾 Tamanho do modelo:              {model_size_kb:.2f} KB")
print(f"🧠 Memória usada pela entrada:     {input_tensor_bytes:.2f} KB")

📊 Comparação por sensor:

🔹 Sensor 1
Referência (Python):  [ 9.605297    2.781259    0.10412411 17.299072  ]
Modelo TFLite      :  [ 9.605291    2.781249    0.10412411 17.299072  ]
Erro absoluto      :  [5.722046e-06 1.001358e-05 0.000000e+00 0.000000e+00]

🔹 Sensor 2
Referência (Python):  [ 9.60675     2.7765203   0.04990168 17.280851  ]
Modelo TFLite      :  [ 9.606744    2.77651     0.04990168 17.280851  ]
Erro absoluto      :  [5.7220459e-06 1.0251999e-05 0.0000000e+00 0.0000000e+00]

🔹 Sensor 3
Referência (Python):  [ 9.605989    2.7803495   0.18566862 17.204725  ]
Modelo TFLite      :  [ 9.605988    2.7803407   0.18566862 17.204725  ]
Erro absoluto      :  [1.9073486e-06 8.8214874e-06 0.0000000e+00 0.0000000e+00]

🔹 Sensor 4
Referência (Python):  [ 9.608006    2.7787597   0.08744715 17.277983  ]
Modelo TFLite      :  [ 9.608012    2.7787492   0.08744715 17.277983  ]
Erro absoluto      :  [6.67572021e-06 1.04904175e-05 0.00000000e+00 0.00000000e+00]

🔹 Sensor 5
Referência (Python)